In [26]:
import pandas as pd
import numpy as np
import math

In [27]:
data = pd.read_csv("AllPOI Simplified.csv", header = None)

In [28]:
left_most = min(data[0])
right_most = max(data[0])
up_most = max(data[1])
down_most = min(data[1])
print("four coord boundary of MBR:",left_most,right_most,up_most,down_most)

four coord boundary of MBR: 115.435043 117.488743 41.029975 39.439945


In [29]:
pointNum = len(data)
pointNum

182323

In [30]:
n = math.log(pointNum / 128, 2) 
if(int(n) % 2 == 0):
    n = int((int(n) + 2) / 2)
else:
    n_begin = int((int(n) + 1) / 2)
print(n)

6


In [31]:
def build_x_grid_index(x, left_most, right_most, grid_num):

    x_unit = (right_most - left_most) / grid_num
    grid_x = int((x - left_most) / x_unit)
    if x == right_most:
        grid_x -= 1
        
    return grid_x

def build_y_grid_index(y, down_most, up_most, grid_num):

    y_unit = (up_most - down_most) / grid_num
    grid_y = int((y - down_most) / y_unit)
    if y == up_most:
        grid_y -= 1
    
    
    return grid_y

def build_grid_index(x, y, left_most, right_most, up_most, down_most, grid_num):

    x_unit = (right_most - left_most) / grid_num
    grid_x = int((x - left_most) / x_unit)
    
    y_unit = (up_most - down_most) / grid_num
    grid_y = int((y - down_most) / y_unit)

    # avoid overflow when y reaches upmost
    if y == up_most:
        grid_y = grid_y - 1

    return int(grid_x + 1 + grid_y * grid_num)

def grid_calculate(n, data):
    left_most = min(data[0])
    right_most = max(data[0])
    up_most = max(data[1])
    down_most = min(data[1])
    
    data_copy = data.copy(deep=True)

    grid_num = 2**n
    
    data_copy['x_gird_index'] = data_copy.apply(lambda p : build_x_grid_index(p[0], left_most,right_most, grid_num), axis = 1)
    data_copy['y_gird_index'] = data_copy.apply(lambda p : build_x_grid_index(p[1], down_most,up_most, grid_num), axis = 1)
    data_copy['grid_index'] = data_copy.apply(lambda p: build_grid_index(p[0], p[1], left_most, right_most, up_most, down_most,grid_num),axis=1)
 
    return data_copy

In [32]:
while(True):
    index_data = grid_calculate(n,data)
    if(max(index_data['grid_index'].value_counts()) > 128):
        n = n + 1
        del index_data
        continue
    else:
        print(n)
        break

9


In [33]:
def more_than_five(n):
    return n > 5
 
tmp = filter(more_than_five, list(index_data['grid_index'].value_counts()))
more_than_five_list = list(tmp)

print(2 ** 18 - len(more_than_five_list))

253779


In [34]:
# SW 左下
# NW 左上
# NE 右上
# SE 右下
left_most = min(data[0])
right_most = max(data[0])
up_most = max(data[1])
down_most = min(data[1])

def generate_SW(x_unit, y_unit, x_gird_index, y_grid_index):
    return (left_most + x_unit*x_gird_index, down_most + y_unit*y_grid_index)
    
def generate_NW(x_unit, y_unit, x_gird_index, y_grid_index):
    return (left_most + x_unit*x_gird_index, down_most + y_unit*(y_grid_index+1))
    
def generate_NE(x_unit, y_unit, x_gird_index, y_grid_index):
    return (left_most + x_unit*(x_gird_index+1), down_most + y_unit*(y_grid_index+1))
    
def generate_SE(x_unit, y_unit, x_gird_index, y_grid_index):
    return (left_most + x_unit*(x_gird_index+1), down_most + y_unit*y_grid_index)
    

In [35]:
left_most = min(data[0])
right_most = max(data[0])
up_most = max(data[1])
down_most = min(data[1])

x_unit = (right_most - left_most) / (2**9)
y_unit = (up_most - down_most) / (2**9)
index_data['SW'] = index_data.apply(lambda p : generate_SW(x_unit, y_unit, p['x_gird_index'],p['y_gird_index']), axis = 1)
index_data['NW'] = index_data.apply(lambda p : generate_NW(x_unit, y_unit, p['x_gird_index'],p['y_gird_index']), axis = 1)
index_data['NE'] = index_data.apply(lambda p : generate_NE(x_unit, y_unit, p['x_gird_index'],p['y_gird_index']), axis = 1)
index_data['SE'] = index_data.apply(lambda p : generate_SE(x_unit, y_unit, p['x_gird_index'],p['y_gird_index']), axis = 1)


In [36]:
index_data.sort_values(by='grid_index')

,0,1,x_gird_index,y_gird_index,grid_index,SW,NW,NE,SE
181676,116.436448,39.439945,249,0,250,"(116.43381507031249, 39.439945)","(116.43381507031249, 39.44305052734375)","(116.437826203125, 39.44305052734375)","(116.437826203125, 39.439945)"
58944,116.407034,39.452780,242,4,2291,"(116.40573714062499, 39.452367109375004)","(116.40573714062499, 39.45547263671875)","(116.4097482734375, 39.45547263671875)","(116.4097482734375, 39.452367109375004)"
58990,116.415178,39.454175,244,4,2293,"(116.41375940625, 39.452367109375004)","(116.41375940625, 39.45547263671875)","(116.4177705390625, 39.45547263671875)","(116.4177705390625, 39.452367109375004)"
112032,116.432675,39.454463,248,4,2297,"(116.4298039375, 39.452367109375004)","(116.4298039375, 39.45547263671875)","(116.43381507031249, 39.45547263671875)","(116.43381507031249, 39.452367109375004)"
76766,116.432219,39.454216,248,4,2297,"(116.4298039375, 39.452367109375004)","(116.4298039375, 39.45547263671875)","(116.43381507031249, 39.45547263671875)","(116.43381507031249, 39.452367109375004)"
...,...,...,...,...,...,...,...,...,...
96262,116.628502,41.007608,297,504,258346,"(116.6263494453125, 41.00513078125)","(116.6263494453125, 41.00823630859375)","(116.63036057812499, 41.00823630859375)","(116.63036057812499, 41.00513078125)"
121597,116.633795,41.029933,298,511,261931,"(116.63036057812499, 41.02686947265625)","(116.63036057812499, 41.029975)","(116.6343717109375, 41.029975)","(116.6343717109375, 41.02686947265625)"
90876,116.633798,41.029928,298,511,261931,"(116.63036057812499, 41.02686947265625)","(116.63036057812499, 41.029975)","(116.6343717109375, 41.029975)","(116.6343717109375, 41.02686947265625)"
90875,116.634111,41.029975,298,511,261931,"(116.63036057812499, 41.02686947265625)","(116.63036057812499, 41.029975)","(116.6343717109375, 41.029975)","(116.6343717109375, 41.02686947265625)"


In [37]:
#Task2
# SW 左下
# NW 左上
# NE 右上
# SE 右下
data_z = data.copy(deep=True)
Z_index = {}
left_most = min(data[0])
right_most = max(data[0])
up_most = max(data[1])
down_most = min(data[1])


def find_part(x, y, x_mid, y_mid):
    if x <= x_mid:
        if y <= y_mid:
            return 'SW'
        else:
            return 'NW'
        
    if x > x_mid:
        if y <= y_mid:
            return 'SE'
        else:
            return 'NE'

def find_mid(x_left, x_right, y_down, y_up):
    x_mid = x_left + (x_right - x_left) / 2
    y_mid = y_down + (y_up - y_down) / 2
    return x_mid, y_mid
    

def dfs(data, resl, x_left, x_right, y_down, y_up):
    #reach resl
    if resl >= 9:
        return
    if len(data) <= 5:
        return
    x_mid, y_mid = find_mid(x_left, x_right, y_down, y_up)
    
    data['part'] = data.apply(lambda p : find_part(p[0], p[1],x_mid,y_mid), axis=1)

    zList = []
    for index, row in data.iterrows():
        if row['part'] == 'SW':
            zList.append(((row[0], row[1]), '1'))
        elif row['part'] == 'NW':
            zList.append(((row[0], row[1]), '2'))
        elif row['part'] == 'SE':
            zList.append(((row[0], row[1]), '3'))
        else:
            zList.append(((row[0], row[1]), '4'))

    zList_undup = list(set(zList))
    
    for z in zList_undup:
        Z_index[z[0]] = Z_index[z[0]] + z[1] if z[0] in Z_index else z[1]
    
    dfs(data[data['part'] == 'SW'], resl+1, x_left, x_mid, y_down, y_mid)
    dfs(data[data['part'] == 'NW'], resl+1, x_left, x_mid, y_mid, y_up)
    dfs(data[data['part'] == 'SE'], resl+1, x_mid, x_right, y_down, y_mid)
    dfs(data[data['part'] == 'NE'], resl+1, x_mid, x_right, y_mid, y_up)

In [38]:
dfs(data_z, 0, left_most, right_most, down_most, up_most)

C:\Users\Jason\AppData\Local\Temp/ipykernel_11400/2513313850.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['part'] = data.apply(lambda p : find_part(p[0], p[1],x_mid,y_mid), axis=1)


In [39]:
Z_index

{(116.608333, 40.187663): '322421311',
 (116.587196, 39.684093): '312134443',
 (116.364224, 39.971912): '143412344',
 (116.408222, 39.925002): '143342231',
 (116.349666, 39.936417): '143322422',
 (116.540729, 40.007374): '321241243',
 (116.547363, 39.907703): '321141423',
 (116.411654, 39.961618): '143431244',
 (116.480255, 39.912076): '321122311',
 (116.419086, 39.616186): '133442313',
 (116.350746, 39.933396): '143322421',
 (116.46858, 39.796739): '312221123',
 (116.207547, 40.564693): '234212121',
 (116.48638, 39.897727): '321121342',
 (116.423725, 39.89103): '143341332',
 (116.404797, 39.912633): '143342113',
 (116.618034, 39.987042): '321421331',
 (116.227581, 40.22808): '144222414',
 (116.39536, 39.99063): '143423334',
 (116.269632, 39.869648): '143132121',
 (116.435966, 39.884204): '143334224',
 (116.316621, 39.947436): '143233144',
 (116.263818, 40.05927): '144113442',
 (116.649634, 39.920283): '321324341',
 (116.228979, 40.21393): '144222314',
 (116.438059, 39.919207): '143344

In [40]:
len(Z_index)

181693

In [41]:
z_max_len = 0
for z_index_value in Z_index.values():
    if(len(z_index_value) > z_max_len):
        z_max_len = len(z_index_value)
print(z_max_len)

9


In [42]:
for key in Z_index:
    if(len(Z_index[key]) < z_max_len):
        add_zero_num = z_max_len - len(Z_index[key])
        Z_index[key] = Z_index[key] + '0' * add_zero_num

In [43]:
Z_index

{(116.608333, 40.187663): '322421311',
 (116.587196, 39.684093): '312134443',
 (116.364224, 39.971912): '143412344',
 (116.408222, 39.925002): '143342231',
 (116.349666, 39.936417): '143322422',
 (116.540729, 40.007374): '321241243',
 (116.547363, 39.907703): '321141423',
 (116.411654, 39.961618): '143431244',
 (116.480255, 39.912076): '321122311',
 (116.419086, 39.616186): '133442313',
 (116.350746, 39.933396): '143322421',
 (116.46858, 39.796739): '312221123',
 (116.207547, 40.564693): '234212121',
 (116.48638, 39.897727): '321121342',
 (116.423725, 39.89103): '143341332',
 (116.404797, 39.912633): '143342113',
 (116.618034, 39.987042): '321421331',
 (116.227581, 40.22808): '144222414',
 (116.39536, 39.99063): '143423334',
 (116.269632, 39.869648): '143132121',
 (116.435966, 39.884204): '143334224',
 (116.316621, 39.947436): '143233144',
 (116.263818, 40.05927): '144113442',
 (116.649634, 39.920283): '321324341',
 (116.228979, 40.21393): '144222314',
 (116.438059, 39.919207): '143344

In [44]:
def make_pos(x,y):
    return (x,y)
index_data.apply(lambda p : generate_SW(x_unit, y_unit, p['x_gird_index'],p['y_gird_index']), axis = 1)
data_z['pos'] = data_z.apply(lambda p : make_pos(p[0],p[1]), axis = 1)
data_z

,0,1,part,pos
0,116.303382,39.908620,SW,"(116.303382, 39.90862)"
1,116.802076,40.335921,NE,"(116.802076, 40.335921)"
2,116.807570,40.362750,NE,"(116.80757, 40.36275)"
3,116.839759,40.371410,NE,"(116.839759, 40.37141)"
4,117.105753,40.140551,SE,"(117.105753, 40.140551)"
...,...,...,...,...
182318,117.240095,40.615747,NE,"(117.240095, 40.615747)"
182319,117.111129,40.213956,SE,"(117.111129, 40.213956)"
182320,116.796412,40.562527,NE,"(116.796412, 40.562527)"
182321,116.459869,39.852378,SW,"(116.459869, 39.852378)"


In [45]:
zvalue_list = []
for index, row in data_z.iterrows():
    point_z_value = Z_index.get(row['pos'])
    zvalue_list.append(point_z_value)

In [46]:
data_z['z_value'] = np.array(zvalue_list)

In [47]:
data_z.sort_values(by="z_value" , ascending=True)

,0,1,part,pos,z_value
96397,115.544369,39.637517,SW,"(115.544369, 39.637517)",111244244
87976,115.544834,39.636930,SW,"(115.544834, 39.63693)",111244244
108179,115.544918,39.637956,SW,"(115.544918, 39.637956)",111244244
104610,115.544665,39.636888,SW,"(115.544665, 39.636888)",111244244
70069,115.544064,39.637424,SW,"(115.544064, 39.637424)",111244244
...,...,...,...,...,...
16782,117.479419,40.648774,NE,"(117.479419, 40.648774)",443333410
58443,117.480715,40.648521,NE,"(117.480715, 40.648521)",443333410
54035,117.479523,40.648382,NE,"(117.479523, 40.648382)",443333410
76474,117.488743,40.649474,NE,"(117.488743, 40.649474)",443333430


In [48]:
data_z = data_z.sort_values(by="z_value" , ascending=True)

In [49]:
data_z

,0,1,part,pos,z_value
96397,115.544369,39.637517,SW,"(115.544369, 39.637517)",111244244
87976,115.544834,39.636930,SW,"(115.544834, 39.63693)",111244244
108179,115.544918,39.637956,SW,"(115.544918, 39.637956)",111244244
104610,115.544665,39.636888,SW,"(115.544665, 39.636888)",111244244
70069,115.544064,39.637424,SW,"(115.544064, 39.637424)",111244244
...,...,...,...,...,...
16782,117.479419,40.648774,NE,"(117.479419, 40.648774)",443333410
58443,117.480715,40.648521,NE,"(117.480715, 40.648521)",443333410
54035,117.479523,40.648382,NE,"(117.479523, 40.648382)",443333410
76474,117.488743,40.649474,NE,"(117.488743, 40.649474)",443333430


In [58]:
def grid_index_win_query(left_down_point, right_up_point, index_data):
    left_most = min(index_data[0])
    right_most = max(index_data[0])
    up_most = max(index_data[1])
    down_most = min(index_data[1])
    x_unit = (right_most - left_most) / 2**9
    y_unit = (up_most - down_most) / 2**9
    res_points = 0
    query_points = 0
    # 1.first calculate all points,some are not in the rec, then we should remove them
    left_down_x_grid = int((left_down_point[0] - left_most)/x_unit)
    left_down_y_grid = int((left_down_point[1] - down_most)/y_unit)
    
    right_up_x_grid = int((right_up_point[0] - left_most)/x_unit)
    right_up_y_grid = int((right_up_point[1] - down_most)/y_unit)
    
    for i in range(left_down_y_grid, right_up_y_grid + 1):
        for j in range(left_down_x_grid, right_up_x_grid + 1):
            res_points = res_points + len(index_data[(index_data['x_gird_index'] == j) & (index_data['y_gird_index'] == i)])
    
    # 2. calculate intersaction gird
    for i in range(left_down_x_grid, right_up_x_grid + 1):
        temp_data_1 = index_data[(index_data['x_gird_index'] == i) & (index_data['y_gird_index'] == left_down_y_grid)]
        query_points = query_points + len(temp_data_1)
        for index, row in temp_data_1.iterrows():
            if row[0] < left_down_point[0] or row[0] > right_up_point[0] or row[1] < left_down_point[1]:
                res_points = res_points - 1
                
    for i in range(left_down_x_grid, right_up_x_grid + 1):
        temp_data_2 = index_data[(index_data['x_gird_index'] == i) & (index_data['y_gird_index'] == right_up_y_grid)]
        query_points = query_points + len(temp_data_2)
        for index, row in temp_data_2.iterrows():
            if row[0] < left_down_point[0] or row[0] > right_up_point[0] or row[1] > right_up_point[1]:
                res_points = res_points - 1
    
        
    for i in range(left_down_y_grid + 1, right_up_y_grid):
        temp_data_3 = index_data[(index_data['x_gird_index'] == left_down_x_grid) & (index_data['y_gird_index'] == i)]
        query_points = query_points + len(temp_data_3)
        for index, row in temp_data_3.iterrows():
            if row[0] < left_down_point[0]:
                res_points = res_points - 1
                
                
    for i in range(left_down_y_grid + 1, right_up_y_grid):
        temp_data_4 = index_data[(index_data['x_gird_index'] == right_up_x_grid) & (index_data['y_gird_index'] == i)]
        query_points = query_points + len(temp_data_4)
        for index, row in temp_data_4.iterrows():
            if row[0] > right_up_point[0]:
                res_points = res_points - 1
                
    return res_points,query_points

In [59]:

def get_single_zvalue(x, y, x1_left, x1_right,y1_up, y1_down):
    single_point_z_value = ''
    for i in range(0,9):
        x_mid = x1_left + (x1_right - x1_left) / 2
        y_mid = y1_down + (y1_up - y1_down) / 2
        part_str = find_part(x, y, x_mid, y_mid)
        
        if part_str == 'SW':
            single_point_z_value += '1'
            x1_right = x_mid
            y1_up = y_mid
        elif part_str == 'NW':
            single_point_z_value += '2'
            x1_right = x_mid
            y1_down = y_mid
        elif part_str == 'SE':
            single_point_z_value += '3'
            x1_left = x_mid
            y1_up = y_mid
        else:
            single_point_z_value += '4'
            x1_left = x_mid
            y1_down = y_mid
            
    return single_point_z_value

def zvalue_index_win_query(left_down_point, right_up_point, data_z):
    left_most = min(data_z[0])
    right_most = max(data_z[0])
    up_most = max(data_z[1])
    down_most = min(data_z[1])

    res_point = 0
    query_point = 0
    
    single_point_z_value_left_down = get_single_zvalue(left_down_point[0],left_down_point[1],left_most,right_most,up_most,down_most)
    single_point_z_value_right_up = get_single_zvalue(right_up_point[0],right_up_point[1],left_most,right_most,up_most,down_most)
    
    tmp_data_z = data_z[(data_z['z_value'] <= single_point_z_value_right_up) & (data_z['z_value'] >= single_point_z_value_left_down)]
    res_point += len(tmp_data_z)
    
    query_point += len(tmp_data_z)
    for index, row in tmp_data_z.iterrows():
        if row[0] < left_down_point[0] or row[0] > right_up_point[0] or row[1] < left_down_point[1] or row[1] > right_up_point[1]:
            res_point -= 1
    return res_point, query_point

In [64]:
import random
left_most = min(data_z[0])
right_most = max(data_z[0])
up_most = max(data_z[1])
down_most = min(data_z[1])
flag = 1

for i in range(0, 25):
    x_max = round(random.uniform(left_most,right_most),6)
    x_min = round(random.uniform(left_most,x_max),6)
    
    y_max = round(random.uniform(down_most,up_most),6)
    y_min = round(random.uniform(down_most,y_max),6)
    
    left_down_point = (x_min, y_min)
    right_up_point = (x_max, y_max)
    
    print('grid_index_win_query')
    
    grid_res_points, grid_query_points = grid_index_win_query(left_down_point,right_up_point,index_data)
    print('result:',grid_res_points)
    print('query',grid_query_points)
    
    print('zvalue_index_win_query')
    
    zvalue_res_points, zvalue_query_points = zvalue_index_win_query(left_down_point,right_up_point,data_z)
    print('result:',zvalue_res_points)
    print('query',zvalue_query_points)
    
        

grid_index_win_query
result: 1700
query 50
zvalue_index_win_query
result: 1700
query 57341
grid_index_win_query
result: 156
query 17
zvalue_index_win_query
result: 156
query 155300
grid_index_win_query
result: 260
query 44
zvalue_index_win_query
result: 260
query 2612
grid_index_win_query
result: 9
query 10
zvalue_index_win_query
result: 9
query 311
grid_index_win_query
result: 234
query 31
zvalue_index_win_query
result: 234
query 154562
grid_index_win_query
result: 0
query 0
zvalue_index_win_query
result: 0
query 359
grid_index_win_query
result: 8
query 19
zvalue_index_win_query
result: 8
query 5305
grid_index_win_query
result: 126741
query 1844
zvalue_index_win_query
result: 126741
query 147306
grid_index_win_query
result: 3052
query 314
zvalue_index_win_query
result: 3052
query 147979
grid_index_win_query
result: 6023
query 193
zvalue_index_win_query
result: 6023
query 13369
grid_index_win_query
result: 271
query 23
zvalue_index_win_query
result: 271
query 122196
grid_index_win_quer